# Import modules

In [1]:
import os
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sklearn
import math
%matplotlib inline
from statsmodels.graphics import tsaplots 

from leap_ec.algorithm import generational_ea
from leap_ec import ops, decoder, representation
from leap_ec.binary_rep import initializers
from leap_ec.binary_rep import problems
from leap_ec.binary_rep.ops import mutate_bitflip

from leap_ec.decoder import IdentityDecoder
import leap_ec

import copy
import random

# Read in data

In [2]:
# Read in the price data
price = pd.read_csv('price.csv')
price = price.drop(columns = 'Date')
price = price.values.tolist()

In [3]:
# Read vector of fuzzy numbers for each trading day
A = pd.read_csv('A.csv')
A = A.drop(columns = 'Date')
A = A.values.tolist()

# Individual class

In [4]:
# This is a function that initializes a genome, used in the individual class
def genome():
    genome = []
    for i in range(0,5):
        row = []
        for j in range (0,4):
            row.append(random.uniform(0, 1))
        genome.append(row)
    return genome

In [5]:
# Individual class
class OurIndividual:
    def __init__(self,genome,decoder,problem,initial_bank):
        self.genome = genome
        self.decoder = decoder
        self.problem = problem
        self.phenome = self.decoder.decode(self.genome)
        self.bank = initial_bank
        ##new feature
        self.fitness = 0
    def clone(self):
        # TODO: It should be a deep copy
        return self
#     def decode(self):
#         return self.phenome
    def evaluate(self, A, price, start, period):
        self.problem.evaluate(self.phenome,A, price, start, period, self.bank)
        # self.bank = self.problem.result_bank
        # return self.problem.result_bank
    
    # # These are class methods
    # @classmethod
    # def create_population(self,n,initialize,decoder,problem,initial_bank):
    #     # Population is a list of individuals
    #     population = []
    #     # Initialize n individuals and put them into the population
    #     # They are with the same decoder and problem but having different genomes
    #     for i in range(0,n):
    #         ind = OurIndividual(initialize(),decoder = decoder, problem = problem, initial_bank = initial_bank)
    #         population.append(ind)
    #     # This function returns a list of individuals of this class type
    #     return population
    
    # @classmethod
    # def evaluate_population(self,population, A, price, start, period):
    #     evaluation = []
    #     for ind in population:
    #         phenome = ind.phenome
    #         fitness = ind.problem.evaluate(phenome, A, price, start, period, ind.bank)
    #         # new feature
    #         self.fitness = fitness
    #         evaluation.append(fitness)
    #     return evaluation

    # @classmethod
    # def get_fitness(self):
    #     return self.fitness

# Population class

In [6]:
# Mutations and crossover
mu, sigma = 0, 0.1 # mean and standard deviation
def mutate(genome):
    # Select the number of mutations that will happen to the matrix
    num_mutation = random.randrange(21)
    for x in range(0,num_mutation):
        # Generate a set of index
        i =  random.randrange(4)
        j =  random.randrange(3)
        
        # Keep the gene positive
        while True:
            # Generate a random  change
            mut = np.random.normal(mu, sigma)
            # Mutate the gene at the index
            genome[i][j] += mut
            if (genome[i][j]>= 0 and genome[i][j] <= 1): break
            else:
                genome[i][j] -= mut 
                continue
    return genome

def crossover(genome):
    # Select the number of crossovers that will happen to the matrix
    num_cross = random.randrange(21)
    for x in range(0,num_cross):
        # Generate 1st set of index
        i_1 =  random.randrange(4)
        j_1 =  random.randrange(3)
        # Generate 2nd set of index
        i_2 =  random.randrange(4)
        j_2 =  random.randrange(3)
        
        temp =  genome[i_1][j_1]
        genome[i_1][j_1] = genome[i_2][j_2]
        genome[i_2][j_2] = temp
    return genome

def one_pt_crossover(genome1, genome2):
    off_spring_1 = pd.DataFrame([])
    off_spring_2 = pd.DataFrame([])
    genome1 = pd.DataFrame(genome1)
    genome2 = pd.DataFrame(genome2)

    cut = random.randint(0,5)
    vec1 = genome1.loc[:,0]
    vec2 = genome2.loc[:,0]
    off_spring_1 = pd.DataFrame(vec1[:cut].append(vec2[cut:]))
    off_spring_2 = pd.DataFrame(vec2[:cut].append(vec1[cut:]))

    for x in range(1,4):
        cut = random.randint(0,5)
        vec1 = genome1.loc[:,x]
        vec2 = genome2.loc[:,x]
        temp_1 = pd.DataFrame(vec1[:cut].append(vec2[cut:]))
        temp_2 = pd.DataFrame(vec2[:cut].append(vec1[cut:]))
        #print('temp1', temp_1)
        off_spring_1 = np.concatenate([off_spring_1, temp_1], axis = 1)
        off_spring_2 = np.concatenate([off_spring_2, temp_2], axis = 1)
    # Change back to lists
    #off_spring_1 = off_spring_1.values.tolist()
    #off_spring_2 = off_spring_2.values.tolist()
    #print(off_spring_1)
    return off_spring_1, off_spring_2


In [7]:
# Population class
class OurPopulation:
    def __init__(self,n,initialize,decoder,problem,initial_bank):
        # Population is a list of individuals
        population = []

        # Initialize n individuals and put them into the population
        # They are with the same decoder and problem but having different genomes
        for i in range(0,n):
            ind = OurIndividual(initialize(),decoder = decoder, problem = problem, initial_bank = initial_bank)
            population.append(ind)
        
        self.population = population
        self.fitness = []
    
    # This evaluation will inherit the individual's previous bank account
    def evaluate(self, A, price, start, period):
        population = self.population
        # Initialize a evaluation score list
        evaluation = []

        # Evaluate the fitness of each individual and append to the list
        for ind in population:
            fitness = ind.problem.evaluate(ind, A, price, start, period, ind.bank)
            ind.fitness = fitness
            evaluation.append(fitness)
        
        self.fitness = evaluation
        # Return the evaluation list
        return evaluation
    
    # This evaluation will evaluate each individual with the same bank
    def evaluate_i(self, A, price, start, period, initial_bank):
        population = self.population
        # Initialize a evaluation score list
        evaluation = []

        # Evaluate the fitness of each individual and append to the list
        for ind in population:
            # Evaluate the individual with the same bank
            fitness = ind.problem.evaluate(ind, A, price, start, period, initial_bank)
            ind.fitness = fitness
            evaluation.append(fitness)
        
        self.fitness = evaluation
        # Return the evaluation list
        return evaluation
    
    # This function evolves the population with each new individual start with initial_bank
    def evolve(self, decoder, problem, price, start, period, initial_bank):
        # initialize
        pop_obj = self
        # population = self.population

        # Get the 2 best performing individuals from orig_population
        # Evaluate the current population with the same initial bank
        pop_eval = pop_obj.evaluate_i(A, price, start, period, initial_bank)
        # The best individual and 2nd best individual
        mom_id = pop_eval.index(max(pop_eval))
        temp = pop_eval
        temp[mom_id] = 0
        dad_id = temp.index(max(temp))
        mom, dad = pop_obj.population[mom_id], pop_obj.population[dad_id]

        # Population is a list of individuals
        new_generation = []
        n = len(pop_obj.population)

        # Initialize n individuals and put them into the population
        # They are with the same decoder and problem but having different genomes
        for i in range(0,math.floor(n/2)):
            gene1, gene2 = one_pt_crossover(mom.genome, dad.genome)
            # Initialize 2 new individuals with initial bank
            ind1 = OurIndividual(gene1,decoder = decoder, problem = problem, initial_bank=initial_bank)
            ind2 = OurIndividual(gene1,decoder = decoder, problem = problem, initial_bank=initial_bank)
            new_generation.append(ind1)
            new_generation.append(ind2)

        # Add the new generation to the original population
        pop_obj.population.extend(new_generation)

        # Evaluate the population that have the original individuals and new individuals
        pop_obj.evaluate(A, price, start, period)

        # Sort the population to get the best performing 40 people that can stay in the population
        population = pop_obj.population
        fitness = pop_obj.fitness
        df = pd.DataFrame({'pop':population,'fit':fitness})
        df.sort_values(by=['fit'])
        population = df['pop'][:n].tolist()
        fitness = df['fit'][:n].tolist()

        pop_obj.population = population
        pop_obj.fitness = fitness
        self = pop_obj
        # sorted_population = (x for _,x in sorted (zip(fitness,population)))
        # zip_obj = zip(fitness,population)
        # sorted_population = sorted(pop_obj.population, key = lambda x:x.fitness)[:n]
        # fitness, population = zip(*sorted(zip(fitness, population)))


        # # This function returns a list of individuals of this class type
        # # return new_population

In [8]:
# ind1 = OurIndividual(genome(),IdentityDecoder(),OurProblem(),initial_bank)
# ind2 = OurIndividual(genome(),IdentityDecoder(),OurProblem(),initial_bank)
# pop = [ind1,ind2]
# fit = [0.3,0.2]
# print(pop)
# sort = [x for _,x in sorted (zip(fit,pop))]
# print(sort)

# Decoder class

After analysis, the decoder class in LEAP intends to generate the phenome only using the genome of the individual. Our initial thoughts was to treat U as the phenome. However, U needed to be calculated combined with every day membership value vectors (A). Thus, our choice of decoder is the `IdentityDecoder()`, which maps the genome into itself.

# Problem class

In [9]:
# Problem class
class OurProblem():
    # This function will return the vector B
    def get_B(self, A, phenome):
        B = []
        for j in range(0,4):
            b = max(min(A[0],phenome[0][j]),
                      min(A[1],phenome[1][j]),
                      min(A[2],phenome[2][j]),
                      min(A[3],phenome[3][j]),
                      min(A[4],phenome[4][j]))
            B.append(b)
        return B
    
     # This function will return the U value
    def get_U(self, B, Lambda):
        top = 0
        bottom = 0
        for i in range(0,4):
            top += B[i]*Lambda[i]
            bottom += Lambda[i]
        U = top/bottom
        return U
    
    # This function will return the signal based on U value
    def get_signal(self, U,upper,lower):        
        if (U > upper): 
            # If U is higher than the upper threshold then buy
            signal = "B"
        elif (U < lower):
            # TODO: not very sure how it works for the sell.....
            # If U is lower than the lower threshold then sell 
            signal = "S"
        else:
            # If else, then no signal was detected
            signal = "N"
        return signal
    
    # This function will return the amount of this deal
    def  get_amount(self, signal, U,upper,lower):
        if (signal == "B"):
            # If buying then the amount is 
            amount = abs(U-upper)
        elif (signal == "S"):
            # If selling then the amount is 
            amount = abs(U-lower)
        else:
            # If no action then no amount
            amount = 0
        
        # Check for maximum limit of amount
        if (amount <= 20):
            pass
        else:
             amount = 20
                
        return amount
    
    # This function will find the price needed for this deal
    def get_price(self, signal,price):
        if (signal == "B"):
            # If buying then at the opening price
            price = price[0]
        elif (signal == "S"):
            # If selling then sell at the closing price
            price = price[4]
        else:
            # If no action then no price
            price = 0
        return price
    
    # This function will return the result of this deal
    def get_result(self, signal, price, amount):
        if (signal == "B"):
            result = -( price * amount )
        elif (signal == "S"):
            result = price * amount
        else:
            result = 0
            
        return result
            
    # This function will evaluate the fitness of a individual
    def evaluate(self, ind, A, price, start, period, initial_bank):
        # Set boundaries for buy and sell
        upper = 0.6
        lower = 0.55
        # Singleton Fuzzifier
        Lambda = [0.25, 0.5, 0.75, 1.0]
        # Get the list of vectors of fuzzy numbers (A)
        A_list = A
        # Get the list of prices with matching index of A (price)
        price_list = price
        
        # Trading start date
        start = start
        # Trading period
        period = period
        # Trading initial bank account value
        result = initial_bank
        
        # Records performance
        upside_perf = []
        downside_perf = []

        # Trade: from start date to the end of the period
        for i in range(start, start+period+1):
            A_previous = A_list[i-1]
            # Calculate the U of the previous trading day
            B = self.get_B(A_previous, ind.phenome)
            U = self.get_U(B, Lambda)
            

            # The signal of this trading day
            signal = self.get_signal(U,upper,lower)
            # The amount of this traidng day
            amount = self.get_amount(signal,U,upper,lower)
            # The price of this deal for this traidng day
            price = self.get_price(signal,price_list[i])
            # The change in bank account of this trading day compounded with this
            daily_result = self.get_result(signal,price,amount) 
            if daily_result > 0:
                upside_perf.append(daily_result/result)
            else:
                downside_perf.append(daily_result/result)
            result += daily_result
        
        ind.bank = result
        
        # Sterling ratio
        # sr_result = (result-initial_bank)/initial_bank/min(downside_perf)

        # Upside Potential Ratio 
        # upr_result = mean(upside_perf)/sqrt(mean(downside_perf)**2 - vars(downside_perf))

        return result
    
    
    # This function will return if the given two are of the same fitness
    def equivalent(self, first_fitness, second_fitness):
        return first_fitness == second_fitness
    
    # This function will return which one of the fitness is better (maximum)
    def better_than(self, first_fitness, second_fitness):
        return max(first_fitness, second_fitness)
        

# Example

In [10]:
# Individual bank test
start = 199 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
total_days = len(A)-period

g = genome()

ind = OurIndividual(g,decoder = IdentityDecoder(), problem = OurProblem(), initial_bank = initial_bank)
# for i in range(0,total_days):
#     start = i
#     ind.bank = initial_bank
#     ind.evaluate(A, price, start, period)
#     print(start, " ", ind.bank)
#ind.evaluate(A, price, start, period)

In [11]:
# Population setup
start = 0 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
n = 40 # Number of people in one population
total_days = len(A)-150 # Obtain the length of the price and A

# Initialize a population
population = OurPopulation(n,genome,IdentityDecoder(),OurProblem(),initial_bank)

# Check the first two individual's genome in this population
print("The genome of the first 2 individual's genome")
print(population.population[0].genome)
print(population.population[1].genome)

The genome of the first 2 individual's genome
[[0.7282217664240963, 0.6311826315936483, 0.4536333582686741, 0.7056456789225337], [0.023213685010533114, 0.046114617539839364, 0.6942255943114448, 0.9821664488030253], [0.523884197785673, 0.9695596394090626, 0.7392275860949338, 0.4736542319114446], [0.007282616486809057, 0.7641757120237498, 0.6249991508204891, 0.4543055547379091], [0.917255478657726, 0.7515104828194474, 0.9303630929704778, 0.331895854004406]]
[[0.7854259540173508, 0.5594591619657105, 0.8212135403844952, 0.49553236874079487], [0.6828316120671295, 0.3748598728011895, 0.2210654564881499, 0.5648338862404786], [0.39191291939590256, 0.9849825810650322, 0.42028799888308843, 0.8747065224825803], [0.8031479088580831, 0.0652781583357267, 0.6721420182638769, 0.8829454819409719], [0.751161595699386, 0.49708560789548095, 0.44795846995634303, 0.9774538230981983]]


In [12]:
# Evaluate a population with inherited bank account
population.evaluate(A, price, start, period)
evaluation = population.fitness

# Check the population's evaluation results
print("The evaluation result of the population")
print(evaluation)

The evaluation result of the population
[61.263120883776104, 44.26051452540954, 61.84524690883978, 92.2926654433515, -9.107720599014474, 76.55409439986292, 110.91723737782849, 212.45501398519966, 88.59386731567652, 199.43551383711738, 128.01443368145252, 95.69740510065067, -8.878697436654468, 127.34500172245056, 97.60737986664968, 131.023997172233, 117.58871677774346, 99.78267489254642, 98.33706676893854, 77.68081586684266, 171.48132135348442, 216.87087151160912, 32.7911123559091, 162.37454697979857, 89.26762593583312, 73.23462218789425, 83.86870781413661, 90.58023743779023, 90.73806032305409, 85.84586406730344, 82.8212964913156, 125.18797889676398, 82.11373607074746, 150.24765369255817, 101.78097271332396, 63.3080048967307, 43.85588866276271, 113.99318241130307, 64.27205738338932, 166.40589462716937]


In [13]:
# Evaluate a population with initial_bank
population.evaluate_i(A, price, start, period, initial_bank)
evaluation = population.fitness

# Check the population's evaluation results
print("The evaluation result of the population")
print(evaluation)

The evaluation result of the population
[61.263120883776104, 44.26051452540954, 61.84524690883978, 92.2926654433515, -9.107720599014474, 76.55409439986292, 110.91723737782849, 212.45501398519966, 88.59386731567652, 199.43551383711738, 128.01443368145252, 95.69740510065067, -8.878697436654468, 127.34500172245056, 97.60737986664968, 131.023997172233, 117.58871677774346, 99.78267489254642, 98.33706676893854, 77.68081586684266, 171.48132135348442, 216.87087151160912, 32.7911123559091, 162.37454697979857, 89.26762593583312, 73.23462218789425, 83.86870781413661, 90.58023743779023, 90.73806032305409, 85.84586406730344, 82.8212964913156, 125.18797889676398, 82.11373607074746, 150.24765369255817, 101.78097271332396, 63.3080048967307, 43.85588866276271, 113.99318241130307, 64.27205738338932, 166.40589462716937]


In [14]:
pop = ["a", "b", "c", "d", "e", "f", "g", "h", "i"]
fit = [ 0,   1,   1,    0,   1,   2,   2,   0,   1]
print(type(pop),type(pop[1]),type(fit),type(fit[1]))

fit, pop = zip(*sorted(zip(fit, pop)))
print(fit)
print(pop)

<class 'list'> <class 'str'> <class 'list'> <class 'int'>
(0, 0, 0, 1, 1, 1, 1, 2, 2)
('a', 'd', 'h', 'b', 'c', 'e', 'i', 'f', 'g')


In [15]:
pop = population.population
fit = population.fitness
print(type(pop),type(pop[1]),type(fit),type(fit[1]))

df = pd.DataFrame({'pop':pop,'fit':fit})
df.sort_values(by=['fit'])
pop = df['pop'].tolist()
fit = df['fit'].tolist()
print(type(pop),type(pop[1]),type(fit),type(fit[1]))

# zip_list = zip(fit, pop)
# fit, pop = sorted(zip_list, key = lambda x:x[1])[:n]
# fit, pop = zip(*sorted(zip(fit, pop)))
# print(fit)
# print(pop)

<class 'list'> <class '__main__.OurIndividual'> <class 'list'> <class 'float'>
<class 'list'> <class '__main__.OurIndividual'> <class 'list'> <class 'float'>


In [16]:
# Make multiple generations
for i in range(0,10):
    start = random.randint(0,total_days)
    population.evolve(IdentityDecoder(),OurProblem(),price,start,period,initial_bank)
    print(start,population.fitness)
    # print(len(population.fitness))
    # print(len(population.population))

4648 [6938.415411297851, 1664.1944615245566, 7605.363633142228, 16300.028620798139, -14669.10066345799, 14957.479542356536, 22819.72694822491, 60564.21739422283, 15611.999349610858, 54896.82199648334, 27865.643778014975, 17111.98253138484, -14502.346623444528, 28725.417175355684, 17459.906652412144, 34988.95102151971, 25538.399359834773, 21096.040086610607, 18153.790210416017, 15419.573255025354, 47294.87465066033, 68039.05051394369, -2360.165179050087, 39431.4515729293, 16873.285109788343, 14644.490286968961, 14508.866934703197, 18103.68940319275, 17051.641132788973, 17731.026432239985, 15283.812423708174, 30109.58217246722, 12477.427273451427, 39903.63143810962, 21381.830571858467, 7769.075919129116, 3479.5942636393524, 22953.132607152136, 8365.611603994505, 50746.58844247639]
3945 [-1832.8534857518446, -5025.340917215835, -1919.2087467480947, 2622.888084771842, -13663.676649365429, 930.3156919295747, 6494.981719329484, 26563.039339173076, 2226.527401870032, 22952.4275841518, 8784.80

## Test

In [17]:
ind = OurIndividual([[0,0.6,0.5,0],
                     [0,0.33,0.33,0.33],
                     [0,0.1,1.0,0.9],
                     [0,0.44,0.5,0.1],
                     [0,0.1,0.2,0.7]],decoder = IdentityDecoder(), problem = OurProblem())

TypeError: __init__() missing 1 required positional argument: 'initial_bank'

In [ ]:
ind_copy = ind.clone()
ind_copy.genome

[[0, 0.6, 0.5, 0],
 [0, 0.33, 0.33, 0.33],
 [0, 0.1, 1.0, 0.9],
 [0, 0.44, 0.5, 0.1],
 [0, 0.1, 0.2, 0.7]]

In [ ]:
ind.phenome

[[0, 0.6, 0.5, 0],
 [0, 0.33, 0.33, 0.33],
 [0, 0.1, 1.0, 0.9],
 [0, 0.44, 0.5, 0.1],
 [0, 0.1, 0.2, 0.7]]

In [ ]:
start = 0
period = 150
initial_bank = 100.0

ind = OurIndividual(genome(),decoder = IdentityDecoder(), problem = OurProblem())

prb = OurProblem()
phenome = ind.phenome
#B = prb.get_B(A[0],phenome)
#B
result = prb.evaluate(phenome, A, price, start, period, initial_bank)
# #ind.evaluate(A, price, start, period, initial_bank)
result

57.67588549683599

In [ ]:
evaluation = ind.evaluate(A, price, start, period, initial_bank)
evaluation

57.67588549683599

In [ ]:
# Population evaluation test
n = 40

population = OurIndividual.create_population(n,genome,IdentityDecoder(),OurProblem())
#person = population[0]
pop_eval = OurIndividual.evaluate_population(population, A, price, start, period, initial_bank)
print(population[0].genome,pop_eval[0])

[[0.5838051997614904, 0.5641701895878842, 0.5128389695028838, 0.6397621141106337], [0.9638862418461862, 0.8596242105555907, 0.4939514396802914, 0.6955909212172895], [0.44773097939880224, 0.9644630865533629, 0.8568435570882578, 0.27957565405573903], [0.7233942160458424, 0.5958716263330441, 0.17741332823687528, 0.30768268160988943], [0.8384733667555498, 0.2659020014798996, 0.46386152964486604, 0.5527486630606854]] 115.70349332300668


KernelInterrupted: Execution interrupted by the Jupyter kernel.

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# one part cross over test
g1 = genome()
g2 = genome()
g3,g4 = one_pt_crossover(g1,g2)
print(g1)
print(g2)
print(g3)
print(g4)
type(g4)

[[0.23103235876728978, 0.5445982617238986, 0.8581571774137974, 0.2886896694358234], [0.7413326821348638, 0.27558180766721163, 0.7885257713501509, 0.7356528954184759], [0.3153184934962304, 0.28926462373279616, 0.35777327122490243, 0.08147742290199456], [0.6858166293898373, 0.167990899187353, 0.13200044412707235, 0.019122244985779857], [0.8854717070961214, 0.0007295300480353317, 0.0861478594852717, 0.4627333314705582]]
[[0.20562791657828472, 0.0616264128989884, 0.07801337131957986, 0.15181448613973225], [0.27343524237226957, 0.3639272062114314, 0.701036550706836, 0.10272435825098614], [0.9039491005087924, 0.264466641357872, 0.41503165345833526, 0.10491642989880046], [0.368498434857241, 0.3641283186728357, 0.23906919377962976, 0.5871058453865341], [0.6652575459927001, 0.48706746207732987, 0.13464718188616365, 0.5513737734885176]]
[[0.23103236 0.54459826 0.85815718 0.28868967]
 [0.74133268 0.27558181 0.78852577 0.7356529 ]
 [0.31531849 0.26446664 0.41503165 0.08147742]
 [0.68581663 0.36412

numpy.ndarray

In [ ]:
# evol test
g1 = genome()
g2 = genome()
# Population setup
start = 0 # start day index
period = 150 # trading period
initial_bank = 100.0 # Initial bank account value
n = 40 # Number of people in one population

# Initialize a population
population = OurIndividual.create_population(n,genome,IdentityDecoder(),OurProblem())
ind1 = OurIndividual(g1, decoder = IdentityDecoder(), problem = OurProblem())
ind2 = OurIndividual(g2, decoder = IdentityDecoder(), problem = OurProblem())


pop2 = evol(ind1,ind2, IdentityDecoder(), OurProblem(),population, price, start, period)


40


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9433f6a9-5256-43e6-8a88-84eded61d4fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>